
# Phase 2 

This notebook **cleans and prepares** the Heart Attack China dataset and WHO dataset and writes the CSV's as follows:
 - analysis: ../data/processed/heart_attack_china_analysis_ready.csv
 - model: ../data/processed/heart_attack_china_model_ready.csv
 - merged: ../data/processed/heart_attack_china_with_who_latest_by_sex.csv

# Raw and output
- RAW: ../data/raw/heart_attack_china.csv
- WHO: ../data/raw/who_health_china.csv
- OUTDIR: ../data/processed




- Clean heart-attack data and engineer features
- Clean WHO data, keep latest year per indicator × sex, and pivot to wide
- Map Gender_simple → WHO sex codes and join
- Save three outputs: analysis-ready, model-ready, and merged with WHO context


## 1) Combining Both Heart_attack_china and who_health_china

In [2]:
RAW_PATH = "../data/raw/heart_attack_china.csv"
WHO_PATH = "../data/raw/who_health_china.csv"
OUTDIR   = "../data/processed"

print("RAW:", RAW_PATH)
print("WHO:", WHO_PATH)
print("OUTDIR:", OUTDIR)

import pandas as pd
from pathlib import Path

# ---------- Cleaning & feature functions  ----------
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[^\w_]", "", regex=True)
    )
    return df

def strip_object_whitespace(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].astype(str).str.strip()
    return df

def map_yes_no(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    yes_no_map = {"yes": 1, "no": 0, "y": 1, "n": 0, "true": 1, "false": 0}
    for c in df.select_dtypes(include="object").columns:
        ser = df[c].astype(str).str.lower()
        mask = ser.isin(yes_no_map.keys())
        if mask.any():
            df.loc[mask, c] = ser[mask].map(yes_no_map).astype("Int64")
    return df

def derive_flags(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

# SBP from Blood_Pressure if present (and SBP not already present)
    if "Blood_Pressure" in df.columns and "SBP" not in df.columns:
        df = df.rename(columns={"Blood_Pressure": "SBP"})
    if "SBP" in df.columns:
        df["SBP"] = pd.to_numeric(df["SBP"], errors="coerce")
        df["SBP_missing"] = df["SBP"].isna()
        df["SBP_hypertensive"] = (df["SBP"] >= 140).astype("Int64")

# Gender_simple: M/F or NA
    if "Gender" in df.columns:
        g = df["Gender"].astype(str).str.upper().str[0]
        g = g.replace({"F": "F", "M": "M"})
        df["Gender_simple"] = g.where(g.isin(["F", "M"]), pd.NA)

# Smoker_flag from Smoking_Status
    if "Smoking_Status" in df.columns:
        s = df["Smoking_Status"].astype(str).str.lower()
        df["Smoker_flag"] = s.isin(["current", "former", "smoker", "heavy"]).astype("Int64")

# Age_band bins
    if "Age" in df.columns:
        df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
        bins = [0, 30, 40, 50, 60, 70, 200]
        labels = ["<30", "30-39", "40-49", "50-59", "60-69", "70+"]
        df["Age_band"] = pd.cut(df["Age"], bins=bins, labels=labels, right=False)

# RiskFactor_count
    candidates = [
        "Hypertension", "Diabetes", "Obesity", "Chronic_Kidney_Disease",
        "Family_History_CVD", "Previous_Heart_Attack"
    ]
    present = [c for c in candidates if c in df.columns]
    for c in present:
        df[c] = pd.to_numeric(df[c], errors="ignore")
        if df[c].dtype == object:
            lc = df[c].astype(str).str.lower()
            df[c] = lc.map({"yes": 1, "no": 0}).astype("Int64")
    if present:
        df["RiskFactor_count"] = (
            df[present].apply(pd.to_numeric, errors="coerce").fillna(0).sum(axis=1).astype("Int64")
        )
    return df

def build_analysis_and_model_ready(df: pd.DataFrame):
    analysis_ready = df.copy()
    keep_cols = [c for c in [
        "Patient_ID","Age","Gender_simple","SBP","SBP_missing","SBP_hypertensive",
        "Smoker_flag","Cholesterol_Level","CVD_Risk_Score","Hypertension","Diabetes",
        "Obesity","RiskFactor_count","Heart_Attack"
    ] if c in df.columns]
    model_ready = df[keep_cols].copy()
    if "Heart_Attack" in model_ready.columns and model_ready["Heart_Attack"].dtype == object:
        model_ready["Heart_Attack"] = (
            model_ready["Heart_Attack"].astype(str).str.lower()
            .map({"yes": 1, "no": 0}).astype("Int64")
        )
    return analysis_ready, model_ready

# ---------- WHO prep: latest-year, sex-matched wide table ----------
def prep_who_latest_by_sex(who_df: pd.DataFrame) -> pd.DataFrame:
    w = normalize_columns(strip_object_whitespace(who_df))

    if "country" in w.columns:
        w = w[w["country"].astype(str).str.lower() == "china"].copy()

# Keep only rows with year, indicator, sex, value
    required = ["year", "indicator", "sex", "value"]
    for col in required:
        if col not in w.columns:
            raise ValueError(f"WHO data missing required column: {col}")

# For each indicator×sex, keep the *latest year* row
    w["year"] = pd.to_numeric(w["year"], errors="coerce")
    w = w.dropna(subset=["year"])
    w = w.sort_values(["indicator", "sex", "year"]).groupby(["indicator", "sex"], as_index=False).tail(1)

# Pivot to wide: columns like WHO_Mean_Blood_Pressure_SEX_MLE_latest
    w["colname"] = (
        "WHO_" + w["indicator"].astype(str).str.replace(r"\s+", "_", regex=True)
        + "_" + w["sex"].astype(str) + "_latest"
    )
    wide = w.pivot_table(index=[], columns="colname", values="value", aggfunc="first")
    wide = wide.reset_index(drop=True)

# Create a small mapping table for sex → WHO sex code (and attach matching columns)
# We'll map M→SEX_MLE, F→SEX_FMLE; BTSX columns (both sexes) can be added to everyone.
# Split wide into three blocks to merge selectively.
    all_cols = wide.columns.tolist()
    btsx_cols = [c for c in all_cols if "_SEX_BTSX_" in c]
    mle_cols  = [c for c in all_cols if "_SEX_MLE_" in c]
    fmle_cols = [c for c in all_cols if "_SEX_FMLE_" in c]

# Build two tiny frames to merge by gender: one for M, one for F (plus BTSX for both)
    wide_m = wide[btsx_cols + mle_cols].copy() if wide.shape[0] else pd.DataFrame(columns=btsx_cols + mle_cols)
    wide_f = wide[btsx_cols + fmle_cols].copy() if wide.shape[0] else pd.DataFrame(columns=btsx_cols + fmle_cols)

# Add a tag column for the join
    if wide_m.shape[0] == 0:
        wide_m = pd.DataFrame([{c: pd.NA for c in (btsx_cols + mle_cols)}])
    if wide_f.shape[0] == 0:
        wide_f = pd.DataFrame([{c: pd.NA for c in (btsx_cols + fmle_cols)}])

    wide_m["__who_gender_simple__"] = "M"
    wide_f["__who_gender_simple__"] = "F"

    who_by_gender = pd.concat([wide_m, wide_f], ignore_index=True)
    return who_by_gender

# ---------- Run the pipeline ----------
# Load
ha_raw = pd.read_csv(RAW_PATH, low_memory=False)
who_raw = pd.read_csv(WHO_PATH, low_memory=False)

# Heart-attack prep
df = strip_object_whitespace(ha_raw)
df = normalize_columns(df)
df = map_yes_no(df)
df = derive_flags(df)

# WHO prep → latest-year, sex-matched wide
who_by_gender = prep_who_latest_by_sex(who_raw)

# Merge: map Gender_simple to WHO block (M/F)
df_merged = df.merge(
    who_by_gender,
    left_on="Gender_simple",
    right_on="__who_gender_simple__",
    how="left"
).drop(columns=["__who_gender_simple__"])

# Build outputs
analysis_ready, model_ready = build_analysis_and_model_ready(df_merged)

# Save
outdir = Path(OUTDIR)
outdir.mkdir(parents=True, exist_ok=True)

paths = {
    "analysis": outdir / "heart_attack_china_analysis_ready.csv",
    "model":    outdir / "heart_attack_china_model_ready.csv",
    "merged":   outdir / "heart_attack_china_with_who_latest_by_sex.csv",
}

analysis_ready.to_csv(paths["analysis"], index=False, encoding="utf-8")
model_ready.to_csv(paths["model"], index=False, encoding="utf-8")
df_merged.to_csv(paths["merged"], index=False, encoding="utf-8")

print("Wrote:")
for k,p in paths.items():
    print(f" - {k}: {p}")


RAW: ../data/raw/heart_attack_china.csv
WHO: ../data/raw/who_health_china.csv
OUTDIR: ../data/processed


/var/folders/3c/wd_jr6ld7xlc29j69x1w1c_40000gn/T/ipykernel_1675/4060674047.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


Wrote:
 - analysis: ../data/processed/heart_attack_china_analysis_ready.csv
 - model: ../data/processed/heart_attack_china_model_ready.csv
 - merged: ../data/processed/heart_attack_china_with_who_latest_by_sex.csv


In [9]:
print("Analysis Ready CSV First 5:")
print()
print(analysis_ready.head(5))
print()
print("Model Ready CSV First 5:")
print()
print(model_ready.head(5))
print()
print("DF Merged CSV First 5:")
print()
print(df_merged.head(5))


Analysis Ready CSV First 5:

   Patient_ID  Age  Gender Smoking_Status  Hypertension  Diabetes  Obesity  \
0           1   55    Male     Non-Smoker             0         0        1   
1           2   66  Female         Smoker             1         0        0   
2           3   69  Female         Smoker             0         0        0   
3           4   45  Female         Smoker             0         1        0   
4           5   39  Female         Smoker             0         0        0   

  Cholesterol_Level Air_Pollution_Exposure Physical_Activity  ...  \
0            Normal                   High              High  ...   
1               Low                 Medium              High  ...   
2               Low                 Medium              High  ...   
3            Normal                 Medium               Low  ...   
4            Normal                 Medium            Medium  ...   

  Heart_Attack SBP_missing SBP_hypertensive  Gender_simple Smoker_flag  \
0            

### With this new merged dataset we can:
- See where the region is most heart attack prone
- What ages they occur in
- What gender they occur in
- What are the contributing factors